In [45]:
import os
import json
import requests
import itertools

import pandas as pd
import seaborn as sns

### 
EIA_API_KEY = os.getenv('EIA_API_KEY')

In [46]:
# uncomment to run responses directly
#url = "http://api.eia.gov/series/?api_key=" + EIA_API_KEY + "&series_id=ELEC.GEN.COW-IL-99.A"

#response = requests.get(url)
#json_data = json.loads(response.text)

In [47]:
# read in electricity generation csv
elec_generation = pd.read_csv('electricity_generation_extractor/us_electric_generation_2001_20.csv')

In [48]:
il_elec_generation = elec_generation.loc[(elec_generation['state']=='IL') & (elec_generation['year']==2020)]

In [49]:
# setting up something to determine what combo of columns == 'all_fuels'
# subset columns
gen_type_columns = []
for col in il_elec_generation.columns:
    if col in ['state', 'year', 'all_fuels']:
        pass
    else: 
        gen_type_columns.append(col)
    

In [59]:
# set target value to iterate through
target = il_elec_generation['all_fuels'].values[0] # set different total based on what you're trying to find
# creating all possible column combinations
combo_list = []
column_sequence = [combo_list.append(list(seq)) for i in range(len(gen_type_columns), 0, -1)
                    for seq in list(itertools.combinations(gen_type_columns, i))]             

In [63]:
# set values to just explore differences
closest_diff = 1000000000 # set obnoxious value that I know is not possible 
closest_combo = None

for combo in combo_list: 
    if il_elec_generation.loc[:, combo].values.sum() == target:
        print("This combo matched the total", combo)
        
    else: 
        if (target - il_elec_generation.loc[:, combo].values.sum() < closest_diff) & (target - il_elec_generation.loc[:, combo].values.sum() >0): 
            closest_diff = target - il_elec_generation.loc[:, combo].values.sum()
            closest_combo = combo
            


In [64]:
print("Combo had this difference from total", closest_diff)
print(closest_combo)

Combo had this difference from total 4.3472299999557436
['coal', 'natural_gas', 'petro_liquids', 'petro_coke', 'nuclear', 'hydro_electric_storage', 'all_solar', 'utility_scale_solar', 'utility_scale_photovoltaic', 'other_renewables', 'geothermal', 'wood_fuels']


In [65]:
il_elec_generation

,state,year,all_fuels,coal,natural_gas,other_gas,petro_liquids,petro_coke,nuclear,hydro_electric,...,wind,utility_scale_solar,utility_scale_photovoltaic,small_scale_photovoltaic,other_renewables,utility_scale_thermal,geothermal,other_biomass,wood_fuels,other
280,IL,2020,173394.5257,31237.85745,24516.13774,284.67241,30.31227,0.0,100246.068,134.981,...,16225.93984,78.94516,78.94516,427.05936,16695.90817,15.577,0.0,391.02316,0.0,248.58871
